# Working with AGOL items - *Streamlined*

---
Fall 2024  
John.Fay@duke.edu

---

In [7]:
#Import basic packages
import pandas as pd

#Import the arcgis "GIS" object and connect to AGOL
from arcgis import GIS
gis = GIS()

#Import other classes
from arcgis.features import FeatureLayer, GeoAccessor

In [8]:
#Fetch the census tract data from its item ID - as before
tracts_item = gis.content.get('20f5d275113e4066bf311236d9dcc3d4')
tracts_item

<Item title:"USA Census Tract Boundaries" type:Feature Layer Collection owner:esri_dm>

In [9]:
#Get its layer
tracts_layer = tracts_item.layers[0]
tracts_layer

<FeatureLayer url:"https://services.arcgis.com/P3ePLMYs2RVChkJx/arcgis/rest/services/USA_Census_Tracts/FeatureServer/0">

In [13]:
#List the fields
[f['name'] for f in tracts_layer.properties.fields]

['OBJECTID',
 'STATE_ABBR',
 'STATE_FIPS',
 'COUNTY_FIPS',
 'STCOFIPS',
 'TRACT_FIPS',
 'FIPS',
 'POPULATION',
 'POP_SQMI',
 'SQMI',
 'POPULATION_2020',
 'POP20_SQMI',
 'Shape__Area',
 'Shape__Length']

In [20]:
#Get the unique values in the STATE_ABBR field
print(tracts_layer.get_unique_values('STATE_ABBR'))

['AK', 'AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'GA', 'HI', 'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA', 'MD', 'ME', 'MI', 'MN', 'MO', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH', 'NJ', 'NM', 'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VA', 'VT', 'WA', 'WI', 'WV', 'WY']


In [22]:
#Filter the layer to a manageable size
tracts_fset = tracts_layer.query("STCOFIPS = '37063'")

In [23]:
#Convert results to a SeDF
tracts_sdf = tracts_fset.sdf
tracts_sdf.head()

,OBJECTID,STATE_ABBR,STATE_FIPS,COUNTY_FIPS,STCOFIPS,TRACT_FIPS,FIPS,POPULATION,POP_SQMI,SQMI,POPULATION_2020,POP20_SQMI,Shape__Area,Shape__Length,SHAPE
0,55572,NC,37,063,37063,001604,37063001604,6831,709.3,9.63,6655,691.1,0.002496,0.226883,"{""rings"": [[[-78.941013719, 36.1238481410001],..."
1,55573,NC,37,063,37063,001801,37063001801,8369,413.7,20.23,8293,409.9,0.005241,0.421162,"{""rings"": [[[-78.8114876779999, 36.09504014], ..."
2,55574,NC,37,063,37063,000301,37063000301,2610,4278.7,0.61,2644,4334.4,0.000159,0.064235,"{""rings"": [[[-78.898370701, 36.017945121], [-7..."
3,55575,NC,37,063,37063,002021,37063002021,4749,2029.5,2.34,4604,1967.5,0.000604,0.124131,"{""rings"": [[[-78.9574567129999, 35.91955509700..."
4,55576,NC,37,063,37063,001712,37063001712,4458,1214.7,3.67,4150,1130.8,0.00095,0.163956,"{""rings"": [[[-78.937917715, 36.0725571300001],..."


In [26]:
tracts_sdf.spatial.plot(
    renderer_type='classedColorRenderer',
    field_name='POP_SQMI',
    classification_method='naturalBreaks',
    num_classes=5,
    color_ramp='Viridis')

MapView(layout=Layout(height='400px', width='100%'))